# Multicontrolled Z decomposition

For creating the Difffusor for the Grover operator is mandatory implement a reflection. The basis of the reflection is Multi-Controlled Z gate. 

QLM allows a direct definition of this kind of multicontrolled gates, but for us will be interesting create a version of the gate using **C-NOT** and one qbit gates (like rotations for example). 

For creating this we are going to use multiplexors but used in a different way. We are going to use the building based of the following references:

* https://quantumcomputing.stackexchange.com/questions/4078/how-to-construct-a-multi-qubit-controlled-z-from-elementary-gates
* https://arxiv.org/abs/quant-ph/0303063
* Schuch, Norbert & Siewert, Jens. (2003). Programmable Networks for Quantum Algorithms. Physical review letters. 91. 027902. 10.1103/PhysRevLett.91.027902. (https://www.researchgate.net/publication/10622559_Programmable_Networks_for_Quantum_Algorithms)

In [ ]:
import sys
sys.path.append("../../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

In [ ]:
from QQuantLib.DL.data_loading import uniform_distribution
#Testing Function. Apply a Uniform distribution and then an input gate
def testing_gate(input_gate):
    """
    Function for testing purpouses. Given a QLM gate creates a uniform distribution based
    on the arity if the inbput gate and apply the gate.
    
    Parameters
    ----------
    
    input_gate : QLM routine 
        QLM routine user want to test
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine for testing input gate. 
    
    
    """
    number_qubits = input_gate.arity
    routine = qlm.QRoutine()
    register = routine.new_wires(number_qubits)    
    routine.apply(uniform_distribution(number_qubits), register)
    routine.apply(input_gate, register)
    return routine

## 1. Outline of the problem

The main idea is convert a multicontrolled Z gate ($C^{n-1}Z$) to a circuit that uses only $CNOT$ and 1 qbit gates (like rotatios or phase gates). A $C^{n-1}Z$ an be seen as a multicontrolled phase of $\pi$: $PH(\pi)$. So our approximation will be implement a $PH(\theta)$ using multiplexor techniques. And then apply $\theta=\pi$.

## 2. Controlled Phase Gate

In order to develop a $C-PH(2\theta)$ following circuit can be used

![title](CZ_multiplexor.png)

The following part of the circuit will be important:

![title](Multiplexor_base.png)

so we are going to create a function only for generating this part: **phase_multiplexor_base**

In [ ]:
@qlm.build_gate("Multiplexor", [float], arity = 2)
def phase_multiplexor_base(theta):
    """
    Implement an initial multiplexor for a controlled phase gate.
    
    Parameters
    ----------
    
    angle : float
        Phase angle to apply
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine wiht the implementation of the basis multiplexor for the controlled phase gate
    
    """
    routine = qlm.QRoutine()
    # This will be a 2-qbits gate
    register = routine.new_wires(2)
    #routine.apply(qlm.CNOT, register[0], register[1])
    routine.apply(qlm.PH(-theta), register[1])
    # Apply the CNOT
    routine.apply(qlm.CNOT, register[0], register[1])
    #Apply the Phase gate (+)
    routine.apply(qlm.PH(theta), register[1])
    return routine

In [ ]:
base_multiplexor = phase_multiplexor_base(np.pi/2.0)
%qatdisplay base_multiplexor --depth 1

The rest part of the circuit will be done by the following function:

In [ ]:
def controlled_phase(angle):
    """
    Implement controlled phase gate using CNOTs and 1 qbit phase gates.
    
    Parameters
    ----------
    
    angle : float
        Phase angle to apply
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine wiht the implementation controlled_phase gate with cnots and 1 qbit phase gates
        
    
    """
    number_qubits = 2
    # We need to divide the input angle for 2
    angle_step = angle/2
    routine = qlm.QRoutine()
    # This will be a 2-qbits gate
    register = routine.new_wires(number_qubits)    
    routine.apply(qlm.PH(angle_step), register[0])
    routine.apply(qlm.CNOT, register[0], register[1])
    base_multiplexor = phase_multiplexor_base(angle_step)
    routine.apply(base_multiplexor, register[0], register[1])
    return routine

In [ ]:
angle = np.pi
controlled_phase_gate = controlled_phase(angle)
print("Controlled Phase Gate")
%qatdisplay controlled_phase_gate
print("Controlled Phase Gate: decomposition")
%qatdisplay controlled_phase_gate --depth 
test_controlled_phase_gate = testing_gate(controlled_phase_gate)
print("Testing Cricuit for Controlled Phase Gate")
%qatdisplay test_controlled_phase_gate
results_controlled_phase_gate, _, _, _, _ = get_results(test_controlled_phase_gate, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
results_controlled_phase_gate

Now we use the default gates from QLM for controlled-phase

In [ ]:
c_phase_qlm = qlm.PH(angle).ctrl(1)
print("QLM Controlled Phase Gate")
%qatdisplay c_phase_qlm
test_c_phase_qlm = testing_gate(c_phase_qlm)
print("Testing Cricuit for QLM Controlled Phase Gate")
%qatdisplay test_c_phase_qlm
results_c_phase_qlm, _, _, _, _ = get_results(test_c_phase_qlm, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
results_c_phase_qlm

In [ ]:
Testing_columns = ['Int_lsb', 'Probability', 'Amplitude']
np.isclose(results_controlled_phase_gate[Testing_columns], results_c_phase_qlm[Testing_columns])

## 3. Recursive Multiplexor creation

We have developed a $C-PH(\theta)$ using $CNOT$ and $PH(\theta)$. Now we need to create the multicontrolled part. For doing this following steps will be applied:
1. Create a new qbit.
2. Apply a CNOT controlled by the before qbit over the new qbit.
3. Take the complete multiplexor construction that was applied on the before qbit and apply to the new qbit..
4. Apply a CNOT that will be controlled by the before qbit and the target will be the created qbit in the step 1.
5. Repeat again the step 3.
6. Steps 2 to 5 create a new multiplexor operator that will be used in the following iteration.

This will be done in an iterative way for the rest of the qbits.

In the folowing graph the before steps are ilustrated

![title](Recursive_Multiplexor.png)

These steps are done in the **recursive_multiplexor** function. The input will be the QLM gate and the before steps will be applied. 

In [ ]:
def recursive_multiplexor(input_gate):
    """
    Create a new multiplexor from an input gate. In this case takes the input gate adds a new qbit and creates
    a new multiplexor by applyingthe input gate a cnot and the input gate again

    Parameters
    ----------
    
    input_gate : QLM routine
        QLM routine with the gate we want for multiplexion
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine with a multiplexion of the input_gate
    
    """
    routine = qlm.QRoutine()
    input_arity = input_gate.arity
    # Create the qbits for the input gate
    old_qbits = routine.new_wires(input_arity)
    # Add a new qbit for multiplexion
    new_qbit = routine.new_wires(1)
    # routine.apply(qlm.CNOT, old_qbits[input_arity-1], new_qbit)
    routine.apply(input_gate, [old_qbits[:input_arity-1], new_qbit])
    routine.apply(qlm.CNOT, old_qbits[input_arity-1], new_qbit)
    routine.apply(input_gate, [old_qbits[:input_arity-1], new_qbit])
    return routine

In [ ]:
%qatdisplay base_multiplexor
print('Base Multiplexor that will be used for the following step')
recursive_base_multiplexor = recursive_multiplexor(base_multiplexor)
print('New multiplexor when one more qbit is added')
%qatdisplay recursive_base_multiplexor --depth  0
print('New multiplexor when one more qbit is added. Decomposition')
%qatdisplay recursive_base_multiplexor --depth  1

## 4. Creation of Multi-Controlled Phase Gate

Function **multiplexor_controlled_z** creates the complete implementation of a multi-controlled phase gate using hte procedure expalined in the before sections. The input fo the function will be:
* angle to phase
* number of qbits for the multi controlled gbate.

**NOTE**
On important question is the the angle of the initial multiplexor. For this procedure will be:

$$\theta_{step} = \frac{\theta}{2^{n-1} }$$

Where: $\theta$ is the deisred phase angle and $n$ the number of qbits of the gate

In [ ]:
@qlm.build_gate("Multiplexor_C_PH", [float, int], arity=lambda x, y: y)
def multiplexor_controlled_ph(angle, number_qubits):
    """
    Multiplexor implementation for a Multi-Controlled-phase gate
    
    Parameters
    ----------
    
    angle : float
        Desired angle for Controlled-Phase application
    number_qubits : int
        Number of qbits for the multi-controlled phase gate 
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine wiht the implementation of a multi-controlled phase gate
    
    """    
    routine = qlm.QRoutine()
    register = routine.new_wires(number_qubits)
    
    angle = angle/(2**(number_qubits-1))
    for i,r in enumerate(register):
        # print('i:', i)
        if i == 0:
            # In the first qbit we need a Phase rotation
            routine.apply(qlm.PH(angle), register[i])
        elif i==1:
            # In the second qbit we need the base gate for the multiplexor
            routine.apply(qlm.CNOT, register[i-1], register[i])
            multiplexor = phase_multiplexor_base(angle)
            # print(register[:i])
            routine.apply(multiplexor, register[:i+1])
        else:
            # For other qbits we need to create the new multiplexor from the before step multiplexor
            routine.apply(qlm.CNOT, register[i-1], register[i])
            multiplexor = recursive_multiplexor(multiplexor)
            routine.apply(multiplexor, register[:i+1])
            
    return routine


In [ ]:
number_of_qbits = 4
angle = np.pi/2.0
multi_plex_ph = multiplexor_controlled_ph(angle, number_of_qbits)
print('Recursive application of multiplexors for mutlicontrolle Phase')
%qatdisplay multi_plex_ph --depth 1
print('Recursive application of multiplexors for mutlicontrolle Phase: Decomposition')
%qatdisplay multi_plex_ph --depth 
test_multi_plex_ph = testing_gate(multi_plex_ph)
print("Testing Multi-Controlled Phase with Multiplexors")
%qatdisplay test_multi_plex_ph
results_multi_plex_ph, _, _, _, _ = get_results(test_multi_plex_ph, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
#·Comparison  with QLM implementation
c_phase_qlm = qlm.PH(angle).ctrl(number_of_qbits-1)
print("QLM Controlled Phase Gate")
%qatdisplay c_phase_qlm
test_c_phase_qlm = testing_gate(c_phase_qlm)
print("Testing Cricuit for QLM Controlled Phase Gate")
%qatdisplay test_c_phase_qlm
results_c_phase_qlm, _, _, _, _ = get_results(test_c_phase_qlm, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
results_multi_plex_ph

In [ ]:
results_c_phase_qlm

In [ ]:
Testing_columns = ['Int_lsb', 'Probability', 'Amplitude']
np.isclose(results_multi_plex_ph[Testing_columns], results_c_phase_qlm[Testing_columns]).all()

## 5. Multi-Controlled-Z

The multi controlled-Z is a particular case of the multi controlled phase where the angle is just $\pi$

In [ ]:
@qlm.build_gate("Multiplexor_C_Z", [int], arity = lambda x: x)
def multiplexor_controlled_z(number_qubits):
    """
    Multiplexor implementation for a multi-controlled-Z gate
    
    Parameters
    ----------
    

    number_qubits : int
        Number of qbits for the multi-controlled phase gate gate
        
    Returns
    _______
    
    routine : QLM routine 
        QLM routine wiht the implementation of a multi-controlled Z gate
    """    
    return multiplexor_controlled_ph(np.pi, number_qubits)

In [ ]:
number_of_qbits = 6
multiplexor_c_c_z = multiplexor_controlled_z(number_of_qbits)
print('Mult Controlled Z gate')
%qatdisplay multiplexor_c_c_z --depth 0
print('Mult Controlled Z gate: Decomposition')
%qatdisplay multiplexor_c_c_z --depth 1
test_multiplexor_c_c_z = testing_gate(multiplexor_c_c_z)
print("Testing Multi-Controlled Z gate with Multiplexors")
%qatdisplay test_multiplexor_c_c_z
results_multiplexor_c_c_z, _, _, _, _ = get_results(test_multiplexor_c_c_z, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
#·Comparison  with QLM implementation
c_Z_qlm = qlm.Z.ctrl(number_of_qbits-1)
print("QLM Controlled Z Gate")
%qatdisplay c_Z_qlm
test_c_Z_qlm = testing_gate(c_Z_qlm)
print("Testing Cricuit for QLM Controlled Phase Gate")
%qatdisplay test_c_Z_qlm
results_c_Z_qlm, _, _, _, _ = get_results(test_c_Z_qlm, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
results_multiplexor_c_c_z

In [ ]:
results_c_Z_qlm

In [ ]:
Testing_columns = ['Int_lsb', 'Probability', 'Amplitude']
np.isclose(results_multiplexor_c_c_z[Testing_columns], results_c_Z_qlm[Testing_columns]).all()